<a href="https://colab.research.google.com/github/gsdev42/Medical-question-summarization-/blob/main/Train_clip_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq accelerate
!pip install -Uqq transformers
import transformers
transformers.__version__
!pip install datasets
!pip install torch torchvision
!pip install Pillow
!pip install requests
!pip install matplotlib

import os
import datasets
from dataclasses import dataclass, field
from typing import Optional
import matplotlib.pyplot as plt
import pandas as pd
import requests
import random
import numpy as np
import torch
from datasets import load_dataset
from PIL import Image
from torchvision.io import ImageReadMode, read_image
from torchvision.transforms import CenterCrop, ConvertImageDtype, Normalize, Resize
from torchvision.transforms.functional import InterpolationMode
from pdb import set_trace

import transformers
from transformers import (
    VisionTextDualEncoderProcessor,
    VisionTextDualEncoderModel,
    AutoImageProcessor,
    AutoModel,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='torchvision')

# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.31.0.dev0")
require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/contrastive-image-text/requirements.txt")


In [2]:
# Define the model and processor
model = VisionTextDualEncoderModel.from_vision_text_pretrained("openai/clip-vit-base-patch32", "roberta-base")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
image_processor = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)

model.save_pretrained("clip-roberta")
processor.save_pretrained("clip-roberta")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

[]

In [5]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict
df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_data.csv')


train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test

dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Question_summ', 'image_path', 'category', 'context', 'description'],
        num_rows: 2412
    })
    test: Dataset({
        features: ['Question', 'Question_summ', 'image_path', 'category', 'context', 'description', 'Unnamed: 6'],
        num_rows: 603
    })
})

In [6]:
# Data arguments
data_args = type('', (), {})()
data_args.image_column = 'image_path'
data_args.caption_column = 'description'
data_args.max_seq_length = 128
data_args.preprocessing_num_workers = 4

In [8]:
# Image transformations
import torchvision.transforms as T
image_transformations = T.Compose([
    T.Resize((224, 224)),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [9]:
# Define tokenize_captions function
def tokenize_captions(examples):
    captions = [example for example in examples[data_args.caption_column]]
    text_inputs = tokenizer(captions, max_length=data_args.max_seq_length, padding="max_length", truncation=True)
    examples["input_ids"] = text_inputs["input_ids"]
    examples["attention_mask"] = text_inputs["attention_mask"]
    return examples


In [10]:
# Define transform_images function
def transform_images(examples):
    images = []
    for image_path in examples[data_args.image_column]:
        image_path = '/content/drive/MyDrive/Multimodal_img/'+ image_path
        try:
            image = Image.open(image_path).convert('RGB')
            image = image_transformations(image)
            images.append(image)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            images.append(torch.zeros((3, 224, 224)))
    examples["pixel_values"] = images
    return examples

In [11]:
# Apply map functions to the datasets
dataset["train"] = dataset["train"].map(
    tokenize_captions,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    load_from_cache_file=False,
    desc="Tokenizing captions for training dataset"
)
dataset["train"] = dataset["train"].map(
    transform_images,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    load_from_cache_file=False,
    desc="Processing images for training dataset"
)

dataset["test"] = dataset["test"].map(
    tokenize_captions,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    load_from_cache_file=False,
    desc="Tokenizing captions for test dataset"
)
dataset["test"] = dataset["test"].map(
    transform_images,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    load_from_cache_file=False,
    desc="Processing images for test dataset"
)


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Tokenizing captions for training dataset (num_proc=4):   0%|          | 0/2412 [00:00<?, ? examples/s]

Processing images for training dataset (num_proc=4):   0%|          | 0/2412 [00:00<?, ? examples/s]

Error processing /content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg'
Error processing /content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg'
Error processing /content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg'
Error processing /content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg'


Tokenizing captions for test dataset (num_proc=4):   0%|          | 0/603 [00:00<?, ? examples/s]

Processing images for test dataset (num_proc=4):   0%|          | 0/603 [00:00<?, ? examples/s]

Error processing /content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Multimodal_img/mouth ulcers_Image_9.jpg'


In [16]:
# Define collate_fn function
def collate_fn(examples):
    pixel_values = torch.stack([torch.tensor(example["pixel_values"]) for example in examples])
    input_ids = torch.tensor([example["input_ids"] for example in examples], dtype=torch.long)
    attention_mask = torch.tensor([example["attention_mask"] for example in examples], dtype=torch.long)
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "return_loss": True,
    }


In [17]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./clip-roberta-finetuned',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    warmup_steps=0,
    weight_decay=0.1,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=10_000,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False
)

In [18]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=processor.tokenizer,
    data_collator=collate_fn,
)

In [19]:
# Train the model
train_result = trainer.train()
trainer.log_metrics("train", train_result.metrics)
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_model("path/to/fine-tuned-model")
processor.save_pretrained("path/to/fine-tuned-model")

Step,Training Loss
10,2.084700
20,1.945400
30,1.887800
40,1.816500
50,1.869400
60,1.805900
70,1.698500
80,1.674700
90,1.709300
100,1.667200


***** train metrics *****
  epoch                    =        3.0
  total_flos               =   899779GF
  train_loss               =     1.2358
  train_runtime            = 0:17:31.99
  train_samples_per_second =      6.878
  train_steps_per_second   =      0.861


***** eval metrics *****
  epoch                   =        3.0
  eval_loss               =     0.7216
  eval_runtime            = 0:01:05.76
  eval_samples_per_second =      9.169
  eval_steps_per_second   =      1.156


[]

In [20]:
from transformers import VisionTextDualEncoderProcessor, VisionTextDualEncoderModel
import torch
from PIL import Image

# Load the fine-tuned model and processor
model = VisionTextDualEncoderModel.from_pretrained('/content/path/to/fine-tuned-model')
processor = VisionTextDualEncoderProcessor.from_pretrained('/content/path/to/fine-tuned-model')

# Define the image path and text description
image_path = '/content/drive/MyDrive/Multimodal_img/swollen eye_Image_2.jpg'
text_description = "A swollen eye is a visible increase in size around the eye area. An image may show puffiness of the eyelids, potentially accompanied by redness, discoloration, or discharge. The swelling can be caused by various factors, including allergies, infections, and injuries, and may be associated with pain, discomfort, or blurred vision."

# Open the image
image = Image.open(image_path).convert('RGB')

# Preprocess the image and text
inputs = processor(images=image, text=text_description, return_tensors="pt", padding=True)

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Extract the image and text embeddings
image_embedding = outputs.image_embeds
text_embedding = outputs.text_embeds

# Compute the cosine similarity between image and text embeddings
cosine_similarity = torch.nn.functional.cosine_similarity(image_embedding, text_embedding)

print(f"Cosine Similarity between image and text: {cosine_similarity.item()}")


Unused or unrecognized kwargs: padding.


Cosine Similarity between image and text: 0.7433799505233765
